In [1]:
import urllib.request
import pandas as pd
import json
from datetime import datetime, timedelta

In [2]:
# Set header
hdr = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}

In [3]:
url = 'https://eservices.mas.gov.sg/api/action/datastore/search.json?resource_id=9a0bf149-308c-4bd2-832d-76c8e6cb47ed&limit=1000&sort=end_of_day%20desc'

req = urllib.request.Request(url,None,hdr)

response = urllib.request.urlopen(req)
data = response.read()

raw_batch1 = json.loads(data.decode())
batch1 = raw_batch1['result']['records']

In [4]:
len(batch1)

1000

In [5]:
batch1[999]['end_of_day']

'2018-09-19'

In [6]:
url = 'https://eservices.mas.gov.sg/api/action/datastore/search.json?resource_id=9a0bf149-308c-4bd2-832d-76c8e6cb47ed&limit=1000&between[end_of_day]=2018-01-01,2018-09-18'

req = urllib.request.Request(url,None,hdr)

response = urllib.request.urlopen(req)
data = response.read()

raw_batch2 = json.loads(data.decode())
batch2 = raw_batch2['result']['records']

In [7]:
len(batch2)

224

In [10]:
ir_init_batch = batch1 + batch2
ir_init_batch

[{'aggregate_volume': None,
  'calculation_method': None,
  'commercial_bills_3m': None,
  'comp_sora_1m': None,
  'comp_sora_3m': None,
  'comp_sora_6m': None,
  'end_of_day': '2022-03-15',
  'highest_transaction': None,
  'interbank_12m': None,
  'interbank_1m': None,
  'interbank_1w': None,
  'interbank_2m': None,
  'interbank_3m': None,
  'interbank_6m': None,
  'interbank_overnight': None,
  'lowest_transaction': None,
  'on_rmb_facility_rate': None,
  'preliminary': 0,
  'published_date': None,
  'sgs_repo_overnight_rate': None,
  'sor_average': None,
  'sora': None,
  'sora_index': None,
  'standing_facility_borrow': '0.80',
  'standing_facility_deposit': '0.00',
  'usd_sibor_3m': None,
  'timestamp': '1647358880'},
 {'aggregate_volume': 1688.0,
  'calculation_method': 'Normal',
  'commercial_bills_3m': None,
  'comp_sora_1m': 0.238,
  'comp_sora_3m': 0.2333,
  'comp_sora_6m': 0.2015,
  'end_of_day': '2022-03-14',
  'highest_transaction': 0.55,
  'interbank_12m': None,
  'interb

In [11]:
len(ir_init_batch)

1224

In [12]:
ir_init_batch[1223]

{'aggregate_volume': None,
 'calculation_method': None,
 'commercial_bills_3m': None,
 'comp_sora_1m': 1.0502,
 'comp_sora_3m': 1.1537,
 'comp_sora_6m': 1.0275,
 'end_of_day': '2018-09-18',
 'highest_transaction': None,
 'interbank_12m': None,
 'interbank_1m': None,
 'interbank_1w': None,
 'interbank_2m': None,
 'interbank_3m': None,
 'interbank_6m': None,
 'interbank_overnight': None,
 'lowest_transaction': None,
 'on_rmb_facility_rate': '3.81',
 'preliminary': 0,
 'published_date': None,
 'sgs_repo_overnight_rate': None,
 'sor_average': 1.2587,
 'sora': 1.2587,
 'sora_index': 0.9809433797,
 'standing_facility_borrow': '1.80',
 'standing_facility_deposit': '0.80',
 'usd_sibor_3m': None,
 'timestamp': '1647358881'}

# Write a function for it

In [22]:
def interest_rate_scraping():
    # Reference to oldest date because 2018-01-01 was PH 
    oldest_datetime_obj = datetime.strptime('2018-01-02', '%Y-%M-%d')
    
    # Obtain latest 1000 rows
    url = 'https://eservices.mas.gov.sg/api/action/datastore/search.json?resource_id=9a0bf149-308c-4bd2-832d-76c8e6cb47ed&limit=1000&sort=end_of_day%20desc'
    req = urllib.request.Request(url,None,hdr)
    response = urllib.request.urlopen(req)
    data = response.read()
    raw_batch1 = json.loads(data.decode())
    batch1 = raw_batch1['result']['records']
    
    ir_init_batch = batch1.copy()
    print(len(ir_init_batch))
    
    # Check if first batch oldest date == 2018-01-02
    curr_old = batch1[999]['end_of_day']
    
    counter = 0
    # Check if oldest date is present within first 1000
    for record in batch1:
        curr_date = record['end_of_day']
        if curr_date == '2018-01-02':
            return (pd.DataFrame(batch1[0:counter+1]))
        else:
            counter += 1
        
    
    while(curr_old != '2018-01-02'):
        # Update new end_date and start date
        new_end = datetime.strptime(curr_old, '%Y-%M-%d') - timedelta(days=1)
        new_start = new_end - timedelta(days=1000)
        
        # If less than 1000 records or just nice 1000 days away, then can just use oldest
        if new_start <= oldest_datetime_obj:
            print("Less than 1000 days from initialisation date")
            date_url = '&between[end_of_day]=2018-01-01,'+ new_end.strftime('%Y-%M-%d')
            curr_old = '2018-01-02'
            
        else:
            print("More than 1000 days from initialisation date")
            date_url = '&between[end_of_day]=' + new_end.strftime('%Y-%M-%d') + ','+ new_end.strftime('%Y-%M-%d')
        
        # Get new requests
        new_url = url + date_url
        curr_req = urllib.request.Request(new_url,None,hdr) # remember to change to headers

        curr_response = urllib.request.urlopen(curr_req)
        curr_data = curr_response.read()

        curr_raw_batch = json.loads(curr_data.decode())
        curr_batch = curr_raw_batch['result']['records']
        
        ir_init_batch = ir_init_batch + curr_batch
        print(len(ir_init_batch))
        
        index = len(curr_batch)
        
        # Update condition:
        if curr_old == '2018-01-02':
            break
        else:
            curr_old = curr_batch[index-1]['end_of_day']


    print("done")
    return ir_init_batch

In [23]:
func_test = interest_rate_scraping()

1000
Less than 1000 days from initialisation date
1229
done


In [25]:
len(func_test)

1229

In [27]:
func_test[0]

{'aggregate_volume': 1967.0,
 'calculation_method': 'Normal',
 'commercial_bills_3m': None,
 'comp_sora_1m': 0.227,
 'comp_sora_3m': 0.2444,
 'comp_sora_6m': 0.2093,
 'end_of_day': '2022-03-22',
 'highest_transaction': 0.75,
 'interbank_12m': None,
 'interbank_1m': None,
 'interbank_1w': None,
 'interbank_2m': None,
 'interbank_3m': None,
 'interbank_6m': None,
 'interbank_overnight': None,
 'lowest_transaction': 0.1,
 'on_rmb_facility_rate': None,
 'preliminary': 0,
 'published_date': '2022-03-23T00:00:00',
 'sgs_repo_overnight_rate': None,
 'sor_average': 0.5122,
 'sora': 0.5122,
 'sora_index': 1.0052926209,
 'standing_facility_borrow': '1.14',
 'standing_facility_deposit': '0.14',
 'usd_sibor_3m': None,
 'timestamp': '1648030096'}

In [29]:
df = pd.DataFrame(func_test)

In [7]:
df

,aggregate_volume,calculation_method,commercial_bills_3m,comp_sora_1m,comp_sora_3m,comp_sora_6m,end_of_day,highest_transaction,interbank_12m,interbank_1m,...,preliminary,published_date,sgs_repo_overnight_rate,sor_average,sora,sora_index,standing_facility_borrow,standing_facility_deposit,usd_sibor_3m,timestamp
0,NaN,None,None,NaN,NaN,NaN,2022-03-18,NaN,None,None,...,0,None,None,NaN,NaN,NaN,1.00,0.00,None,1647637057
1,1833.0,Normal,None,0.2400,0.2353,0.2039,2022-03-17,0.65,None,None,...,0,2022-03-18T00:00:00,None,0.2828,0.2828,1.005245,0.99,0.00,None,1647637057
2,2098.0,Normal,None,0.2400,0.2349,0.2031,2022-03-16,0.45,None,None,...,0,2022-03-17T00:00:00,None,0.3080,0.3080,1.005237,0.85,0.00,None,1647637057
3,2358.0,Normal,None,0.2398,0.2336,0.2021,2022-03-15,0.45,None,None,...,0,2022-03-16T00:00:00,None,0.2256,0.2256,1.005228,0.80,0.00,None,1647637057
4,1688.0,Normal,None,0.2380,0.2333,0.2015,2022-03-14,0.55,None,None,...,0,2022-03-15T00:00:00,None,0.1697,0.1697,1.005222,0.73,0.00,None,1647637057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,NaN,None,None,0.8557,0.8474,0.7604,2018-01-05,NaN,None,None,...,0,None,None,0.3425,0.3425,0.974626,1.14,0.14,None,1647637060
1222,NaN,None,None,0.8557,0.8474,0.7604,2018-01-05,NaN,None,None,...,0,None,None,0.3425,0.3425,0.974626,1.14,0.14,None,1647637060
1223,NaN,None,None,0.8967,0.8545,0.7730,2018-01-04,NaN,None,None,...,0,None,None,0.6058,0.6058,0.974599,1.50,0.50,None,1647637060
1224,NaN,None,None,0.9092,0.8519,0.7763,2018-01-03,NaN,None,None,...,0,None,None,1.0355,1.0355,0.974582,2.16,1.16,None,1647637060


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   aggregate_volume           392 non-null    float64
 1   calculation_method         407 non-null    object 
 2   commercial_bills_3m        0 non-null      object 
 3   comp_sora_1m               1223 non-null   float64
 4   comp_sora_3m               1223 non-null   float64
 5   comp_sora_6m               1223 non-null   float64
 6   end_of_day                 1224 non-null   object 
 7   highest_transaction        392 non-null    float64
 8   interbank_12m              0 non-null      object 
 9   interbank_1m               0 non-null      object 
 10  interbank_1w               0 non-null      object 
 11  interbank_2m               0 non-null      object 
 12  interbank_3m               0 non-null      object 
 13  interbank_6m               0 non-null      objec

In [8]:
df = df[df['comp_sora_1m'].notna()]
df

,aggregate_volume,calculation_method,commercial_bills_3m,comp_sora_1m,comp_sora_3m,comp_sora_6m,end_of_day,highest_transaction,interbank_12m,interbank_1m,...,preliminary,published_date,sgs_repo_overnight_rate,sor_average,sora,sora_index,standing_facility_borrow,standing_facility_deposit,usd_sibor_3m,timestamp
1,1833.0,Normal,None,0.2400,0.2353,0.2039,2022-03-17,0.65,None,None,...,0,2022-03-18T00:00:00,None,0.2828,0.2828,1.005245,0.99,0.00,None,1647637057
2,2098.0,Normal,None,0.2400,0.2349,0.2031,2022-03-16,0.45,None,None,...,0,2022-03-17T00:00:00,None,0.3080,0.3080,1.005237,0.85,0.00,None,1647637057
3,2358.0,Normal,None,0.2398,0.2336,0.2021,2022-03-15,0.45,None,None,...,0,2022-03-16T00:00:00,None,0.2256,0.2256,1.005228,0.80,0.00,None,1647637057
4,1688.0,Normal,None,0.2380,0.2333,0.2015,2022-03-14,0.55,None,None,...,0,2022-03-15T00:00:00,None,0.1697,0.1697,1.005222,0.73,0.00,None,1647637057
5,2054.0,Normal,None,0.2364,0.2351,0.2009,2022-03-11,0.25,None,None,...,0,2022-03-14T00:00:00,None,0.1339,0.1339,1.005217,0.76,0.00,None,1647637057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,NaN,None,None,0.8557,0.8474,0.7604,2018-01-05,NaN,None,None,...,0,None,None,0.3425,0.3425,0.974626,1.14,0.14,None,1647637060
1222,NaN,None,None,0.8557,0.8474,0.7604,2018-01-05,NaN,None,None,...,0,None,None,0.3425,0.3425,0.974626,1.14,0.14,None,1647637060
1223,NaN,None,None,0.8967,0.8545,0.7730,2018-01-04,NaN,None,None,...,0,None,None,0.6058,0.6058,0.974599,1.50,0.50,None,1647637060
1224,NaN,None,None,0.9092,0.8519,0.7763,2018-01-03,NaN,None,None,...,0,None,None,1.0355,1.0355,0.974582,2.16,1.16,None,1647637060


In [10]:
deprecated_cols = ['interbank_overnight', 'interbank_1w', 'interbank_1m', 'interbank_2m',
                  'interbank_3m', 'interbank_6m', 'interbank_12m', 'commercial_bills_3m',
                  'sgs_repo_overnight_rate']

In [13]:
# Step 1 is to remove deprecated columns
df = df.drop(deprecated_cols, axis=1)

In [15]:
df.head(5)

,aggregate_volume,calculation_method,comp_sora_1m,comp_sora_3m,comp_sora_6m,end_of_day,highest_transaction,lowest_transaction,on_rmb_facility_rate,preliminary,published_date,sor_average,sora,sora_index,standing_facility_borrow,standing_facility_deposit,usd_sibor_3m,timestamp
1,1833.0,Normal,0.2400,0.2353,0.2039,2022-03-17,0.65,0.03,None,0,2022-03-18T00:00:00,0.2828,0.2828,1.005245,0.99,0.00,None,1647637057
2,2098.0,Normal,0.2400,0.2349,0.2031,2022-03-16,0.45,0.01,None,0,2022-03-17T00:00:00,0.3080,0.3080,1.005237,0.85,0.00,None,1647637057
3,2358.0,Normal,0.2398,0.2336,0.2021,2022-03-15,0.45,0.01,None,0,2022-03-16T00:00:00,0.2256,0.2256,1.005228,0.80,0.00,None,1647637057
4,1688.0,Normal,0.2380,0.2333,0.2015,2022-03-14,0.55,0.08,None,0,2022-03-15T00:00:00,0.1697,0.1697,1.005222,0.73,0.00,None,1647637057
5,2054.0,Normal,0.2364,0.2351,0.2009,2022-03-11,0.25,0.03,None,0,2022-03-14T00:00:00,0.1339,0.1339,1.005217,0.76,0.00,None,1647637057


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1225 entries, 1 to 1225
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   aggregate_volume           395 non-null    float64
 1   calculation_method         410 non-null    object 
 2   comp_sora_1m               1225 non-null   float64
 3   comp_sora_3m               1225 non-null   float64
 4   comp_sora_6m               1225 non-null   float64
 5   end_of_day                 1225 non-null   object 
 6   highest_transaction        395 non-null    float64
 7   lowest_transaction         395 non-null    float64
 8   on_rmb_facility_rate       891 non-null    object 
 9   preliminary                1225 non-null   int64  
 10  published_date             410 non-null    object 
 11  sor_average                1225 non-null   float64
 12  sora                       1225 non-null   float64
 13  sora_index                 1225 non-null   float

Step 2 is to ensure that the number of non-null in sora same as length of total values

In [27]:
# Step 3 is to replace all null values through backfill
df = df.bfill(axis=1)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1225 entries, 1 to 1225
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   aggregate_volume           1225 non-null   object
 1   calculation_method         1225 non-null   object
 2   comp_sora_1m               1225 non-null   object
 3   comp_sora_3m               1225 non-null   object
 4   comp_sora_6m               1225 non-null   object
 5   end_of_day                 1225 non-null   object
 6   highest_transaction        1225 non-null   object
 7   lowest_transaction         1225 non-null   object
 8   on_rmb_facility_rate       1225 non-null   object
 9   preliminary                1225 non-null   object
 10  published_date             1225 non-null   object
 11  sor_average                1225 non-null   object
 12  sora                       1225 non-null   object
 13  sora_index                 1225 non-null   object
 14  standing

In [58]:
# Write Function for cleaning
def clean_interest(input_df):
    deprecated_cols = ['interbank_overnight', 'interbank_1w', 'interbank_1m', 'interbank_2m',
                  'interbank_3m', 'interbank_6m', 'interbank_12m', 'commercial_bills_3m',
                  'sgs_repo_overnight_rate', 'usd_sibor_3m']
    pass_clean = False
    
    # Step 1 is to remove deprecated columns
    input_df = input_df.drop(deprecated_cols, axis=1)
    
    # Step 2 is check if end_of_day got null 
    if (input_df['end_of_day'].isnull().values.any() == True):
        print("Re-download Data! Data is faulty...")
        return pass_clean
    
    else:
        if (input_df.isnull().values.any() == True):
            # Step 3 is to bfill na values
            input_df = input_df.fillna(method='ffill')
            input_df = input_df.iloc[1:]
        print("Basic Data Cleaning Complete.")
    
    return input_df

In [59]:
output = clean_interest(df)

Basic Data Cleaning Complete.


In [62]:
output.dtypes

aggregate_volume             float64
calculation_method            object
comp_sora_1m                 float64
comp_sora_3m                 float64
comp_sora_6m                 float64
end_of_day                    object
highest_transaction          float64
lowest_transaction           float64
on_rmb_facility_rate          object
preliminary                    int64
published_date                object
sor_average                  float64
sora                         float64
sora_index                   float64
standing_facility_borrow      object
standing_facility_deposit     object
timestamp                     object
dtype: object